In [1]:
import pandas, IPython.core.display

In [2]:
src = pandas.ExcelFile('data/real/2017/CAM data from iPads/2017 CAM data from iPads.xlsx')

In [3]:
src.sheet_names

['2017 CAM data Erl',
 'schema (WIP reverse engineer)',
 '2017 CAM iPad data Tyler',
 'Combined iPad 2017 CAM data']

In [4]:
sheets_to_process = {sheet_name.split(' ')[-1]: src.parse(sheet_name) for sheet_name in src.sheet_names
    if any([person_name in sheet_name for person_name in ['Erl', 'Tyler']])}

In [5]:
sheets_to_rename = [sheet for sheet in sheets_to_process.values()
                    if any([str(column).find(' ') for column in sheet.columns])]

In [6]:
for sheet_to_rename in sheets_to_rename:
    sheet_to_rename.rename(mapper=lambda x: str(x).split(' ')[0], axis='columns', inplace=True)

In [7]:
df = pandas.concat(sheets_to_process, names=['Sheet','Row'])

In [8]:
import collections

In [9]:
class NestedDefaultdict(collections.defaultdict):
    data = None
    name = None
    parent_key = None

    def __init__(self):
        collections.defaultdict.__init__(self, NestedDefaultdict)

In [10]:
coltree = NestedDefaultdict()
for column in df.columns:
    pointer = coltree
    pointer.parent_key = ''
    for word in str(column).split('__'):
        pointer = pointer[word]
        pointer.parent_key = word
    pointer.name = column
    pointer.data = df[column]
coltree.default_factory = None

In [11]:
def visualize_tree(node):
    """Given a dictionary, display a list of keys, and all keys 
    of any contained dictionaries, recursively. Indent to show depth
    of recursion."""

    def visualize_node(node, depth=0):
        for key, value in node.items():
            output.append(f"{'.   ' * depth}{key}"
                          f"{'/' if value.keys() else ''}"
                          f"{' : ' + str(value.data.count()) if value.data is not None else ''}"
                         )
            visualize_node(value, depth + 1)

    output = []
    visualize_node(node)
    return '\n'.join(output)

In [12]:
def get_descendant_columns(node):
    return_value = []
    if node.data is not None:
        return_value.append(node)
    for child in node.values():
        return_value.extend(get_descendant_columns(child))
    return return_value

In [13]:
def get_descendant_column_names(node):
    return [column.name for column in get_descendant_columns(node)]

In [14]:
def get_descendant_frame(node, frame=None):
    if frame is None:
        return pandas.DataFrame({column.name: column.data for column in get_descendant_columns(node)})
    return frame[get_descendant_column_names(node)]

In [15]:
def get_child_columns(node):
    return_value = []
    if node.data is not None:
        return_value.append(node)
    for child in node.values():
        return_value.append(child)
    return return_value

In [16]:
def get_child_column_names(node):
    return [column.name for column in get_child_columns(node)]

In [17]:
def get_child_frame(node, frame=None):
    if frame is None:
        return pandas.DataFrame({column.name: column.data for column in get_child_columns(node)})
    return frame[get_child_column_names(node)]

In [18]:
def convert_datetime(frame):
    output_frame = frame.copy()
    foplt = 'fields__oSets__oPoints__location__timestamp'
    output_frame[foplt] = pandas.to_datetime(frame[foplt], unit='ms')
    for column in ['fields__oSets__date', 'fields__date']:
        output_frame[column] = pandas.to_datetime(output_frame[column], infer_datetime_format=True)
    return output_frame

In [20]:
df = convert_datetime(df)

In [21]:
df[['fields__oSets__date','fields__date', 'fields__oSets__oPoints__location__timestamp']].dropna(how='all')

fields__oSets__date            fields__date  \
Sheet Row                                                    
Erl   0    2017-08-02 13:12:09.542                     NaT   
      70   2017-08-09 09:25:11.710 2017-08-09 09:24:11.845   
      84                       NaT                     NaT   
      98                       NaT                     NaT   
      112                      NaT                     NaT   
      126                      NaT                     NaT   
      140  2017-08-09 10:06:25.480 2017-08-09 10:01:29.326   
      154                      NaT                     NaT   
      168                      NaT                     NaT   
      182                      NaT                     NaT   
      196                      NaT                     NaT   
      210  2017-08-09 11:21:01.555 2017-08-09 11:16:15.922   
      224                      NaT                     NaT   
      238                      NaT                     NaT   
      252                      NaT                     NaT   
      266                      NaT                     NaT   
      280                      NaT                     NaT   
      294                      NaT                     NaT   
      308                      NaT                     NaT   
      336                      NaT                     NaT   
      350  2017-08-09 11:37:20.862 2017-08-09 11:17:15.791   
      364                      NaT                     NaT   
      378                      NaT                     NaT   
      392                      NaT                     NaT   
      406                      NaT                     NaT   
      420                      NaT                     NaT   
      434                      NaT                     NaT   
      448                      NaT                     NaT   
      462                      NaT                     NaT   
      476                      NaT                     NaT   
...                            ...                     ...   
Tyler 2884                     NaT                     NaT   
      2898                     NaT                     NaT   
      2926                     NaT                     NaT   
      2940 2017-08-25 12:54:07.934                     NaT   
      3080 2017-08-01 13:01:17.968 2017-08-01 12:59:56.247   
      3094                     NaT                     NaT   
      3108                     NaT                     NaT   
      3122                     NaT                     NaT   
      3136                     NaT                     NaT   
      3150 2017-08-25 13:05:14.062                     NaT   
      3290 2017-08-10 14:22:57.492 2017-08-10 14:21:41.648   
      3360 2017-08-18 15:31:02.572                     NaT   
      3500 2017-08-22 16:01:49.995                     NaT   
      3528                     NaT                     NaT   
      3542                     NaT                     NaT   
      3556                     NaT                     NaT   
      3570                     NaT                     NaT   
      3584                     NaT                     NaT   
      3598                     NaT                     NaT   
      3612                     NaT                     NaT   
      3626                     NaT                     NaT   
      3640 2017-08-10 14:37:29.806 2017-08-10 14:36:39.936   
      3654                     NaT                     NaT   
      3668                     NaT                     NaT   
      3682                     NaT                     NaT   
      3696                     NaT                     NaT   
      3710 2017-08-22 15:42:36.395 2017-08-22 15:41:43.544   
      3724                     NaT                     NaT   
      3752                     NaT                     NaT   
      3766                     NaT                     NaT   

           fields__oSets__oPoints__location__timestamp  
Sheet Row                                          

In [25]:
get_child_frame(coltree['fields']).dropna(how='all')

NaN  fields__crop             fields__date  \
Sheet Row                                                 
Erl   70    None   CROPS.WHEAT  2017-08-09T09:24:11.845   
      140   None   CROPS.WHEAT  2017-08-09T10:01:29.326   
      210   None  CROPS.BARLEY  2017-08-09T11:16:15.922   
      350   None    CROPS.OATS  2017-08-09T11:17:15.791   
      490   None   CROPS.WHEAT  2017-08-10T14:27:04.887   
      560   None   CROPS.WHEAT  2017-08-17T11:10:41.881   
      700   None   CROPS.WHEAT  2017-08-17T13:05:51.122   
      840   None   CROPS.WHEAT  2017-08-22T16:01:41.018   
Tyler 0     None   CROPS.WHEAT  2017-07-14T12:30:31.587   
      350   None  CROPS.BARLEY  2017-07-18T10:46:49.163   
      770   None    CROPS.OATS  2017-07-18T11:35:17.050   
      1190  None   CROPS.WHEAT  2017-07-18T14:07:55.743   
      1610  None  CROPS.BARLEY  2017-07-18T14:39:49.012   
      2030  None  CROPS.BARLEY  2017-07-19T10:21:45.036   
      2240  None   CROPS.WHEAT  2017-07-19T10:41:49.541   
      2520  None   CROPS.WHEAT  2017-07-19T14:07:06.447   
      2870  None   CROPS.WHEAT  2017-08-01T11:35:22.572   
      3080  None   CROPS.WHEAT  2017-08-01T12:59:56.247   
      3290  None   CROPS.WHEAT  2017-08-10T14:21:41.648   
      3640  None   CROPS.WHEAT  2017-08-10T14:36:39.936   
      3710  None   CROPS.WHEAT  2017-08-22T15:41:43.544   

                                                 fields__desc   fields__image  \
Sheet Row                                                                       
Erl   70                      Midge susceptible wheat. Awned.   img/wheat.png   
      140               Early planted awned midge susceptible   img/wheat.png   
      210              Next to SEF Wheat and near faba beans   img/barley.png   
      350                  Next to SEF peas and a wheat field    img/oats.png   
      490                                   Kernen cover crop   img/wheat.png   
      560   Wheat 2 by canola and soybeans but on other si...   img/wheat.png   
      700                         Next to soybeans and canola   img/wheat.png   
      840                                                 NaN   img/wheat.png   
Tyler 0                                                   NaN   img/wheat.png   
      350                             Right next to SEF wheat  img/barley.png   
      770                  Next to SEF Peas, near the canola     img/oats.png   
      1190                                                NaN   img/wheat.png   
      1610                                                NaN  img/barley.png   
      2030                        Right next to Alvena Canola  img/barley.png   
      2240                 Futher down the road towards Wakaw   img/wheat.png   
      2520                                   Next to treeline   img/wheat.png   
      2870                                                NaN   img/wheat.png   
      3080   Wheat 1 is next to the faba beans and soybeans I   img/wheat.png   
      3290          Research field cover crop. Student trial.   img/wheat.png   
      3640                                                NaN   img/wheat.png   
      3710                                Closer to compound    img/wheat.png   

                  fields__name  
Sheet Row                       
Erl   70     Llewellyn wheat 1  
      140            Llewellyn  
      210           SEF Barley  
      350             SEF Oats  
      490         Kernen wheat  
      560        Outlook wheat  
      700      Outlook wheat 1  
      840       Kernan Wheat-1  
Tyler 0              Sef wheat  
      350          SEF Barley   
      770             SEF Oats  
      1190    Llewelyn Wheat-1  
      1610     Llewelyn Barley  
      2030       Alvena Barley  
      2240        Alvena Wheat  
      2520       Melfort Wheat  
      2870     Outlook Wheat-2  
      3080     Outlook Wheat-1  
      3290        Kernen wheat  
      3640  Kernen cover wheat  
      3710      Kernan wheat 2